In [1]:
# Import all packages used
import pandas as pd
import numpy as np
import spacy as sp
import string
import en_core_web_sm
import nltk as nl
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn import svm

from sklearn.ensemble import RandomForestClassifier


from sklearn.naive_bayes import BernoulliNB

model = svm.SVC()
param = {'kernel': ['linear','rbf']}

lemm_obj = sp.load('en_core_web_lg')
lemm_obj.max_length = 1500000
ps = nl.PorterStemmer()

vect_cnt = CountVectorizer(analyzer='word',min_df=1,stop_words='english',token_pattern='[a-zA-Z0-9]{1,}')
tfidf_vect = TfidfVectorizer(analyzer = 'word', min_df = 1, stop_words = 'english',vocabulary=20000, token_pattern = '[a-zA-Z0-9]{1,}')

allowed_postags = ['NOUN','ADJ','VERB','ADV']

import pandas as pd
from collections import deque
import random
import numpy as np
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import time
from sklearn import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import IPython

C:\Users\ragha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ragha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ragha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ragha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
gen_df_encode = pd.read_pickle("D:\Machine Learning\Vamsi Thesis\Code\Results\gen_df_encode.pkl")

In [3]:
fullMovieData = pd.read_pickle(r"D:\Machine Learning\Vamsi Thesis\Code\Results\fullMovieData.pkl")
# List of genres
g_list = fullMovieData.apply(lambda x: x['genres'].split('|'),axis = 1)
g_list=g_list.to_list()
g_list=list(set(sum(g_list,[])))
g_list

['Animation',
 'IMAX',
 'Fantasy',
 'Crime',
 'Musical',
 'Horror',
 'Thriller',
 'Sci-Fi',
 'Film-Noir',
 'Children',
 'Mystery',
 'War',
 'Action',
 'Drama',
 'Western',
 'Comedy',
 'Romance',
 'Documentary',
 'Adventure']

In [4]:
Final_data=pd.DataFrame(columns=["Genre","Actual Positives","Actual Negatives","Vectorizer","Model","Test_Precision","Test_Recall","Test_Fscore","Precision","Recall","Fscore"])
Final_data.shape
#Master function SVM RF Naive Bayes
def prediction(df,text, genre, vectorizer, model, param):
    result=[]
    print("Genre for prediction: ", genre)
    result.append(genre)
    
    #Sampling
    df_model_0 = df[df[genre] == 0]
    df_model_1 = df[df[genre] == 1]
    print("Number of 0s: ", df_model_0.shape[0])
    print("Number of 1s: ", df_model_1.shape[0])
    result.append(df_model_0.shape[0])
    result.append(df_model_1.shape[0]) 
    
    sample_size = 2*df_model_1.shape[0]
    if sample_size > len(df_model_0) : sample_size = len(df_model_0)
    df_model_s=pd.concat([df_model_1,df_model_0.sample(sample_size)])
    print("Shape of the Sampled DataFrame: ", df_model_s.shape)
    #result.append(df_model_s.shape[0])
    
    #Train-Test Split
    X_df = df_model_s[text]
    y_df = df_model_s[genre]
    X_train,X_test,y_train,y_test = train_test_split(X_df, y_df, test_size = 0.25, random_state = 1,
                                                     shuffle = True)
    
    #Vectorization
    print("\n")
    print("Vectorization Start")
    vect = vectorizer(analyzer = 'word', min_df = 1, stop_words = 'english', token_pattern = '[a-zA-Z]{2,}')
    vect_fit = vect.fit(X_train)
    result.append(vectorizer)
    
    X_train_vec = pd.DataFrame(vect_fit.transform(X_train).toarray())
    X_train_vec.columns = vect_fit.get_feature_names()
    
    X_test_vec = pd.DataFrame(vect_fit.transform(X_test).toarray())
    X_test_vec.columns = vect_fit.get_feature_names()
    print("Vectorization End")
    
    #Model
    print("\n")
    print("Model Building Start")
    model = model
    param = param
    result.append(model)
    
    gs = GridSearchCV(model, param, cv = 5,n_jobs=2)
    gs_fit = gs.fit(X_train_vec, y_train)
    
    test_pred_cnt = pd.DataFrame(gs.predict(X_test_vec))
    train_pred_cnt = pd.DataFrame(gs.predict(X_train_vec))
    print("Model Building End")

    # Accuracy metrics

    #Test Accuracy
    print("\n")
    print("Sample data Test Set Confusion Matrix: ")
    print(confusion_matrix(y_test, test_pred_cnt))
    precision, recall, fscore, train_support = score(y_test, test_pred_cnt, pos_label = 1, average = 'binary')
    print("Sample data Test Set Precision: ", precision)
    print("Sample data Test Set Recall: ", recall)
    print("Sample data Test Set FScore: ", fscore)
    print("\n")
    result.extend([precision, recall, fscore])

    # Train Accuracy
    print("Sample data Train Set Confusion Matrix: ")
    print(confusion_matrix(y_train, train_pred_cnt))
    precision, recall, fscore, train_support = score(y_train, train_pred_cnt, pos_label = 1, average = 'binary')
    print("Sample data Train Set (Precision, Recall, Fscore) :",precision, recall, fscore)
    print("\n")

    #Accuracy on full Data
    X_df_full = df[text]
    y_df_full = df[genre]

    # vectorizer
    X_full_tf = pd.DataFrame(vect_fit.transform(X_df_full).toarray())

    print("Probability Evaluation and Concat Start")
    full_pred_cnt = pd.DataFrame(gs.predict(X_full_tf))
    full_pred_prob = pd.DataFrame(gs.predict_proba(X_full_tf))
    df = pd.concat([df,full_pred_prob], axis = 1)
    df.rename(columns ={1:genre+'_prob'},inplace=True)
    df.drop([0],axis=1,inplace=True)
    print("Probability Evaluation and Concat End")
    
    print("\n")
    print("Full Dataset Confusion Matrix " , model)
    print(confusion_matrix(y_df_full, full_pred_cnt))
    precision, recall, fscore, train_support = score(y_df_full, full_pred_cnt, pos_label = 1, average = 'binary')
    print("Full Dataset Precision: ", precision)
    print("Full Dataset Recall: ", recall)
    print("Full Dataset FScore: ", fscore)
    print("\n")
    result.extend([precision, recall, fscore])
    
    print("Appending results to Final_data Start")
    Final_data.loc[len(Final_data)]=result
    print("Appending results to Final_data End")
    return df

In [5]:
start = time.time()
df_test=gen_df_encode
for i in g_list:    
    df_test = prediction(df_test,'plot_lemm',i,TfidfVectorizer, svm.SVC(probability=True),  {'kernel': ['linear','rbf']})
df_test.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\SVM_Probability_Distribution_plots.pkl')
Result.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\Result_SVM.pkl')
end = time.time()
total_time = end-start

print("Total Time taken:" + str(end-start))

Genre for prediction:  Animation
Number of 0s:  9998
Number of 1s:  404
Shape of the Sampled DataFrame:  (1212, 29)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[187  13]
 [ 43  60]]
Sample data Test Set Precision:  0.821917808219178
Sample data Test Set Recall:  0.5825242718446602
Sample data Test Set FScore:  0.6818181818181818


Sample data Train Set Confusion Matrix: 
[[607   1]
 [  3 298]]
Sample data Train Set (Precision, Recall, Fscore) : 0.9966555183946488 0.9900332225913622 0.9933333333333334


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  SVC(probability=True)
[[9492  506]
 [  46  358]]
Full Dataset Precision:  0.41435185185185186
Full Dataset Recall:  0.8861386138613861
Full Dataset FScore:  0.5646687697160883


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  IMAX
Number of 0s:  10231

Full Dataset Confusion Matrix  SVC(probability=True)
[[9582  792]
 [   2   26]]
Full Dataset Precision:  0.03178484107579462
Full Dataset Recall:  0.9285714285714286
Full Dataset FScore:  0.061465721040189124


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Children
Number of 0s:  9884
Number of 1s:  518
Shape of the Sampled DataFrame:  (1554, 38)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[256  13]
 [ 42  78]]
Sample data Test Set Precision:  0.8571428571428571
Sample data Test Set Recall:  0.65
Sample data Test Set FScore:  0.7393364928909953


Sample data Train Set Confusion Matrix: 
[[767   0]
 [  7 391]]
Sample data Train Set (Precision, Recall, Fscore) : 1.0 0.9824120603015075 0.991128010139417


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  SVC(probability=True)
[[9286  598]
 [  49  46

Sample data Test Set FScore:  0.8544061302681991


Sample data Train Set Confusion Matrix: 
[[1594    8]
 [  11  772]]
Sample data Train Set (Precision, Recall, Fscore) : 0.9897435897435898 0.9859514687100894 0.9878438899552143


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  SVC(probability=True)
[[8986  356]
 [  65  995]]
Full Dataset Precision:  0.7364914877868246
Full Dataset Recall:  0.9386792452830188
Full Dataset FScore:  0.8253836582330982


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Adventure
Number of 0s:  9516
Number of 1s:  886
Shape of the Sampled DataFrame:  (2658, 47)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[403  43]
 [ 85 134]]
Sample data Test Set Precision:  0.7570621468926554
Sample data Test Set Recall:  0.6118721461187214
Sample data Test Set FScore:  0.676767676767

NameError: name 'Result' is not defined

In [ ]:
start = time.time()
df_test=gen_df_encode
for i in g_list:    
    df_test = prediction(df_test,'plot_tt',i,TfidfVectorizer, svm.SVC(probability=True),  {'kernel': ['linear','rbf']})
df_test.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\SVM_Probability_Distribution_tt.pkl')
Result.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\Result_SVM.pkl')
end = time.time()
total_time = end-start

print("Total Time taken:" + str(end-start))